# Use `tokenator` to limit costs of your AI projects

A problem that I often face : I get a cool project idea which involves using LLMs or building some kind of an AI agent. Once it's built, I want to share it with the world so that everyone can use my app and see how cool it is. 

But there's a problem. Running an AI project has real costs. The more people use it, the more money I need to spend powering the product. In such a case, people usually end up posting the code on github, making a video/blog or end up using a cheaper LLM and sacrificing accuracy. More substantial project integrate Auth. 

But not anymore!

With `tokenator`, you can publish your AI project online for the world to see and set a daily cap of how much money would you like to spend on your AI project. The lucky and early access users will be able to access the project without hiccups. If people aren't able to access it today, ask them to checkout the code, your blog or comeback tomorrow!

Similar to other `tokenator` cookbooks, we are going to be analyzing a W2 tax form and understanding how much tax this person has to pay for the year 2024.

In this example, we will be doing some rate limiting using `tokenator` to control costs.

Let's start with installing some dependencies. I am going to be using the openai and anthropic models to complete this task.

In [ ]:
%pip install tokenator openai anthropic --quiet

Next, we will setup 2 functions to access our model. This is done so that we don't have to initialize our openai and anthropic clients repeatedly. 

We will be using the structured outputs mode of openai to first get structured data out of the W2 form image and then use either openai models or anthropic models to check whether the person should get a refund.

In [11]:
# Add API Keys here
OPENAI_API_KEY="api-key"
ANTHROPIC_API_KEY="api-key"

from tokenator import tokenator_openai, tokenator_anthropic

from openai import OpenAI
from anthropic import Anthropic

def call_openai_structured(system_prompt, user_prompt, image_url, model="gpt-4o", execution_id=None, response_format=None):
    client = tokenator_openai(OpenAI(api_key=OPENAI_API_KEY))

    messages = [
        {"role": "system", "content": system_prompt},
        {
            "role": "user",
            "content": [
                {"type": "text", "text": user_prompt},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": image_url,
                    },
                },
            ],
        }
    ]

    response = client.beta.chat.completions.parse(
        model=model,
        messages=messages,
        response_format=response_format,
        execution_id=execution_id,
    )

    return response

def call_openai_unstructured(system_prompt, user_prompt, model="gpt-4o"):
    client = tokenator_openai(OpenAI(api_key=OPENAI_API_KEY))

    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "system", "content": system_prompt},
                  {"role": "user", "content": user_prompt}],
    )

    return response

def call_anthropic(system_prompt, user_prompt, model="claude-3-5-sonnet-20241022"):
    client = tokenator_anthropic(Anthropic(api_key=ANTHROPIC_API_KEY))

    response = client.messages.create(
        model=model,
        system=system_prompt,
        messages=[{"role": "user", "content": user_prompt}],
        max_tokens=5000,
    )

    return response


Before we build our AI agent, let's build a cost limiter(similar to rate limiter) using `tokenator`.

In [13]:
from tokenator import usage

def cost_limiter(cost_limit_dollars) -> bool:
    usage_last_day = usage.last_day()
    if usage_last_day.total_cost > cost_limit_dollars:
        return False
    else:
        return True

In the above function, we use tokenator to get the total usage of the last day.
Then, we use the total_cost field to compare with the cost limit set for the project.

Alternatively, you can also use tokenator's other functions below to set limits hourly, weekly or monthly.

```python
usage.last_hour()
usage.last_day()
usage.last_week()
usage.last_month()
```

Let's build our AI agent.
Here are the steps : 
- Before we involve any AI models, we will first use our `cost_limiter()` function to check whether ew have breached the daily cost limit. If no, we execute the agent, otherwise, we print a message and return. In a real app, you could throw an Exception which can be caught upstream.
- Get a W2 form image
- Pass it to OpenAI gpt 4o and get some income and tax related details
- Use these details and ask a model whether this person should be getting a tax refund or not

In [14]:
from pydantic import BaseModel

def execute_agent(image_url, task1_model="gpt-4o", task2_provider="openai", task2_model="gpt-4o", cost_limit_dollars=10):

    if not cost_limiter(cost_limit_dollars):
        print("Cost limit exceeded. Please try again tomorrow.")
        return
    
    # pydantic class for structured output
    class W2Details(BaseModel):
        name: str
        taxation_state: str
        income: float
        federal_tax_paid: float
        state_tax_paid: float
        social_security_tax_paid: float
        medicare_tax_paid: float

    user_prompt = f"Extract the following details from the W2 form image : {image_url}"

    system_prompt = "You are an expert at extracting details from W2 forms. You are given an image of a W2 form and you need to extract the fields required in the output"

    # 1st LLM call
    response = call_openai_structured(
        system_prompt=system_prompt, 
        user_prompt=user_prompt, 
        image_url=image_url, 
        model=task1_model, 
        response_format=W2Details
    )

    print("Extracted details from W2 form : ")
    print(response.choices[0].message.content)


    # ------------------------------------------------------------------------------------------------

    # Now, we will use the extracted details to ask a model whether this person should be getting a tax refund or not
    user_prompt = f"Based on the following details, determine if this person should be getting a tax refund or not : {response.choices[0].message.content}"

    system_prompt = "You are an expert at determining whether a person should be getting a tax refund or not. You are given a set of details and you need to determine if this person should be getting a tax refund or not. Assume no deductions or credits are applied to the income. Reply concisely."

    # 2nd LLM call
    if task2_provider == "openai":
        response = call_openai_unstructured(
            system_prompt, 
            user_prompt, 
            model=task2_model
        )
        print("Refund status : ")
        print(response.choices[0].message.content)

    elif task2_provider == "anthropic":
        response = call_anthropic(
            system_prompt, 
            user_prompt, 
            model=task2_model
        )
        print("Refund status : ")
        print(response.content[0].text)


Let's run our agent in a loop to see if our rate limiting works or not.
Just to simulate a cost limiting case, we will keep an artificially low cost limit of 1 cent.

In [16]:
image_url = "https://www.patriotsoftware.com/wp-content/uploads/2024/03/2024-Form-W-2-1.png"

for i in range(10):
    print(f"Running agent {i+1} of 10")
    execute_agent(image_url=image_url, cost_limit_dollars=0.05)
    print("--------------------------------")


Running agent 1 of 10
Extracted details from W2 form : 
{"name":"Abby L Smith","taxation_state":"OH","income":50000.00,"federal_tax_paid":4092.00,"state_tax_paid":1040.88,"social_security_tax_paid":3100.00,"medicare_tax_paid":725.00}
Refund status : 
To determine if Abby L Smith should get a tax refund, let's evaluate the taxes:

1. **Federal Tax**: 
    - 2023 Federal Income Tax Rates for a single filer:
      - 10% on income up to $11,000
      - 12% on income from $11,001 to $44,725
      - 22% on income from $44,726 to $95,375

   Calculating Abby's federal tax:
   - 10% of $11,000 = $1,100
   - 12% of ($44,725 - $11,001) = $4,047
   - 22% of ($50,000 - $44,725) = $1,159.50
   - Total federal tax: $1,100 + $4,047 + $1,159.50 = $6,306.50

   Abby paid $4,092.00, so she overpaid by $6,306.50 - $4,092.00 = $2,214.50.

2. **State Tax**: 
   - Ohio uses a progressive tax system, but without specific brackets/rates, assume Abby's payment aligns roughly with her liability.

Abby L Smith s